In [10]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.linear_model import Ridge

In [2]:
data_train = pd.read_csv('geocoding.csv', index_col=0)
data_train.head()

,country,home_add_lat,home_add_lon,work_add_lat,work_add_lon,spend_lat,spend_lon,city_lat,city_lon
0,1.0,59.851,30.232,59.847,30.177,59.844072,30.179153,59.939095,30.315868
1,1.0,59.851,30.232,59.847,30.177,59.844072,30.179153,59.939095,30.315868
2,1.0,59.851,30.232,59.847,30.177,59.858198,30.229024,59.939095,30.315868
3,1.0,59.851,30.232,59.847,30.177,59.844072,30.179153,59.939095,30.315868
4,1.0,59.851,30.232,59.847,30.177,59.844072,30.179153,59.939095,30.315868


In [3]:
data_test = pd.read_csv('test_df.csv', index_col=0)
data_test = data_test.drop(['pos_address_lat'], axis=1)
data_test = data_test.dropna()
data_test.head()

/home/nikita/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,customer_id,spend_lat,spend_lon,city_lat,city_lon
0,00fd410f5c580c8351cafa88d82b60f3,55.967487,37.913682,37.475000,55.702000
1,00fd410f5c580c8351cafa88d82b60f3,55.971296,37.905184,37.475000,55.702000
2,00fd410f5c580c8351cafa88d82b60f3,56.016591,37.909098,56.010428,37.847155
3,00fd410f5c580c8351cafa88d82b60f3,55.964507,37.937912,37.475000,55.702000
4,00fd410f5c580c8351cafa88d82b60f3,55.776802,37.657352,55.755814,37.617635


In [4]:
y_home = data_train[['home_add_lat', 'home_add_lon']]
y_work = data_train[['home_add_lat', 'home_add_lon']]
X = data_train[['spend_lat', 'spend_lon', 'city_lat', 'city_lon']]
y_home = np.array(y_home)
y_work = np.array(y_work)
X = np.array(X)

In [11]:
clf_home = Ridge(alpha=1.0)
clf_home.fit(X, y_home)

clf_work = Ridge(alpha=1.0)
clf_work.fit(X, y_work)

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [12]:
X_test = data_test[['spend_lat', 'spend_lon', 'city_lat', 'city_lon']]
X_test = X_test.dropna()
X_test = np.array(X_test)
answers_home = clf_home.predict(X_test)
answers_work = clf_work.predict(X_test)

In [13]:
home_lat = []
home_lon = []
work_lat = []
work_lon = []

for answer in answers_home:
    lat, lon = answer
    home_lat.append(lat)
    home_lon.append(lon)

for answer in answers_work:
    lat, lon = answer
    work_lat.append(lat)
    work_lon.append(lon)
    
home_lat = pd.Series(home_lat)
home_lon = pd.Series(home_lon)
work_lat = pd.Series(work_lat)
work_lon = pd.Series(work_lon)

In [14]:
data_test['work_add_lat'] = work_lat.values
data_test['work_add_lon'] = work_lon.values
data_test['home_add_lat'] = home_lat.values
data_test['home_add_lon'] = home_lon.values
data_test.head()

,customer_id,spend_lat,spend_lon,city_lat,city_lon,work_add_lat,work_add_lon,home_add_lat,home_add_lon
0,00fd410f5c580c8351cafa88d82b60f3,55.967487,37.913682,37.475000,55.702000,50.809718,46.287872,50.809718,46.287872
1,00fd410f5c580c8351cafa88d82b60f3,55.971296,37.905184,37.475000,55.702000,50.811159,46.285452,50.811159,46.285452
2,00fd410f5c580c8351cafa88d82b60f3,56.016591,37.909098,56.010428,37.847155,56.260057,38.232057,56.260057,38.232057
3,00fd410f5c580c8351cafa88d82b60f3,55.964507,37.937912,37.475000,55.702000,50.808660,46.293875,50.808660,46.293875
4,00fd410f5c580c8351cafa88d82b60f3,55.776802,37.657352,55.755814,37.617635,56.100338,38.019737,56.100338,38.019737


In [15]:
final = data_test[['customer_id', 'work_add_lat', 'work_add_lon', 'home_add_lat', 'home_add_lon']]
final.head()

,customer_id,work_add_lat,work_add_lon,home_add_lat,home_add_lon
0,00fd410f5c580c8351cafa88d82b60f3,50.809718,46.287872,50.809718,46.287872
1,00fd410f5c580c8351cafa88d82b60f3,50.811159,46.285452,50.811159,46.285452
2,00fd410f5c580c8351cafa88d82b60f3,56.260057,38.232057,56.260057,38.232057
3,00fd410f5c580c8351cafa88d82b60f3,50.808660,46.293875,50.808660,46.293875
4,00fd410f5c580c8351cafa88d82b60f3,56.100338,38.019737,56.100338,38.019737


In [16]:
from collections import Counter
customers = pd.unique(final.customer_id)
work_add_lat = []
work_add_lon = []
home_add_lat = []
home_add_lon = []
i = 0
fin = len(customers)
for customer in customers:    
    df_cus = final[final.customer_id == customer]    
    work_add_lat.append( Counter(df_cus['work_add_lat']).most_common(1)[0][0] )
    work_add_lon.append( Counter(df_cus['work_add_lon']).most_common(1)[0][0] )
    home_add_lat.append( Counter(df_cus['home_add_lat']).most_common(1)[0][0] )
    home_add_lon.append( Counter(df_cus['home_add_lon']).most_common(1)[0][0] )
    i+=1
    print(i / fin *100)
    
print( len(customers), len(work_add_lon), len(work_add_lat), len(home_add_lon), len(home_add_lat) )

0.01000300090027008
0.02000600180054016
0.030009002700810243
0.04001200360108032
0.05001500450135041
0.060018005401620486
0.07002100630189056
0.08002400720216064
0.09002700810243072
0.10003000900270081
0.1100330099029709
0.12003601080324097
0.13003901170351106
0.14004201260378113
0.15004501350405122
0.1600480144043213
0.17005101530459138
0.18005401620486144
0.19005701710513154
0.20006001800540163
0.2100630189056717
0.2200660198059418
0.23006902070621185
0.24007202160648194
0.250075022506752
0.26007802340702213
0.2700810243072922
0.28008402520756226
0.2900870261078323
0.30009002700810244
0.3100930279083725
0.3200960288086426
0.3300990297089127
0.34010203060918276
0.3501050315094528
0.3601080324097229
0.370111033309993
0.3801140342102631
0.3901170351105332
0.40012003601080326
0.4101230369110733
0.4201260378113434
0.43012903871161345
0.4401320396118836
0.45013504051215364
0.4601380414124237
0.4701410423126938
0.4801440432129639
0.49014704411323395
0.500150045013504
0.5101530459137742
0.52

4.431329398819646
4.441332399719916
4.451335400620186
4.461338401520456
4.471341402420726
4.481344403320996
4.491347404221266
4.501350405121537
4.511353406021806
4.521356406922076
4.531359407822347
4.541362408722617
4.551365409622886
4.561368410523157
4.571371411423427
4.581374412323697
4.5913774132239675
4.601380414124237
4.611383415024507
4.6213864159247775
4.631389416825048
4.641392417725317
4.6513954186255875
4.661398419525858
4.671401420426128
4.681404421326398
4.691407422226668
4.701410423126938
4.711413424027208
4.721416424927479
4.731419425827749
4.741422426728018
4.751425427628289
4.761428428528559
4.771431429428829
4.7814344303290985
4.791437431229369
4.801440432129639
4.811443433029909
4.82144643393018
4.831449434830449
4.841452435730719
4.85145543663099
4.86145843753126
4.871461438431529
4.8814644393318
4.89146744023207
4.90147044113234
4.9114734420326105
4.92147644293288
4.93147944383315
4.9414824447334205
4.95148544563369
4.96148844653396
4.97149144743423
4.9814944483345


9.012703811143343
9.022706812043612
9.032709812943883
9.042712813844153
9.052715814744424
9.062718815644693
9.072721816544965
9.082724817445234
9.092727818345505
9.102730819245773
9.112733820146044
9.122736821046313
9.132739821946585
9.142742822846854
9.152745823747125
9.162748824647394
9.172751825547664
9.182754826447935
9.192757827348204
9.202760828248474
9.212763829148745
9.222766830049014
9.232769830949286
9.242772831849555
9.252775832749824
9.262778833650096
9.272781834550365
9.282784835450634
9.292787836350906
9.302790837251175
9.312793838151446
9.322796839051716
9.332799839951985
9.342802840852256
9.352805841752525
9.362808842652797
9.372811843553066
9.382814844453335
9.392817845353605
9.402820846253876
9.412823847154145
9.422826848054417
9.432829848954686
9.442832849854957
9.452835850755227
9.462838851655498
9.472841852555765
9.482844853456037
9.492847854356306
9.502850855256577
9.512853856156847
9.522856857057118
9.532859857957387
9.542862858857658
9.552865859757928
9.56286886

13.45403621086326
13.464039211763529
13.4740422126638
13.48404521356407
13.494048214464339
13.50405121536461
13.514054216264878
13.52405721716515
13.534060218065418
13.544063218965691
13.554066219865959
13.564069220766232
13.5740722216665
13.584075222566769
13.59407822346704
13.60408122436731
13.61408422526758
13.62408722616785
13.634090227068121
13.64409322796839
13.654096228868662
13.664099229768931
13.674102230669199
13.684105231569472
13.69410823246974
13.704111233370012
13.71411423427028
13.724117235170553
13.73412023607082
13.744123236971092
13.754126237871361
13.76412923877163
13.774132239671902
13.784135240572171
13.794138241472442
13.804141242372712
13.814144243272983
13.824147244173252
13.834150245073523
13.844153245973793
13.85415624687406
13.864159247774332
13.874162248674601
13.884165249574872
13.894168250475142
13.904171251375413
13.914174252275682
13.924177253175953
13.934180254076223
13.944183254976492
13.954186255876763
13.964189256777033
13.974192257677304
13.98419525

17.855356606982095
17.865359607882365
17.875362608782634
17.885365609682903
17.895368610583176
17.905371611483446
17.915374612383715
17.925377613283985
17.935380614184254
17.945383615084527
17.955386615984796
17.965389616885066
17.975392617785335
17.985395618685608
17.995398619585874
18.005401620486147
18.015404621386416
18.025407622286686
18.035410623186955
18.045413624087224
18.055416624987497
18.065419625887767
18.075422626788036
18.085425627688306
18.09542862858858
18.105431629488848
18.115434630389117
18.125437631289387
18.135440632189656
18.14544363308993
18.155446633990195
18.165449634890468
18.175452635790737
18.18545563669101
18.195458637591276
18.205461638491546
18.21546463939182
18.225467640292088
18.235470641192357
18.245473642092627
18.2554766429929
18.26547964389317
18.27548264479344
18.285485645693708
18.295488646593977
18.30549164749425
18.315494648394516
18.32549764929479
18.33550065019506
18.345503651095328
18.355506651995597
18.36550965289587
18.37551265379614
18.385

...

81.18435530659198
81.19435830749224
81.20436130839252
81.21436430929279
81.22436731019306
81.23437031109333
81.2443733119936
81.25437631289387
81.26437931379414
81.2743823146944
81.28438531559468
81.29438831649495
81.30439131739521
81.31439431829548
81.32439731919577
81.33440032009604
81.34440332099629
81.35440632189656
81.36440932279683
81.37441232369711
81.38441532459738
81.39441832549765
81.40442132639792
81.41442432729818
81.42442732819846
81.43443032909873
81.444433329999
81.45443633089927
81.46443933179954
81.47444233269981
81.48444533360008
81.49444833450035
81.50445133540062
81.51445433630089
81.52445733720116
81.53446033810144
81.54446333900171
81.55446633990196
81.56446934080223
81.5744723417025
81.58447534260279
81.59447834350306
81.60448134440333
81.6144843453036
81.62448734620386
81.63449034710413
81.6444933480044
81.65449634890467
81.66449934980494
81.67450235070521
81.68450535160548
81.69450835250575
81.70451135340602
81.71451435430629
81.72451735520656
81.73452035610683

85.79573872161649
85.80574172251676
85.81574472341703
85.8257477243173
85.83575072521757
85.84575372611783
85.8557567270181
85.86575972791837
85.87576272881864
85.88576572971891
85.89576873061918
85.90577173151945
85.91577473241972
85.92577773331999
85.93578073422027
85.94578373512054
85.95578673602081
85.96578973692107
85.97579273782134
85.98579573872162
85.99579873962189
86.00580174052216
86.01580474142243
86.0258077423227
86.03581074322297
86.04581374412324
86.0558167450235
86.06581974592378
86.07582274682404
86.08582574772431
86.0958287486246
86.10583174952485
86.11583475042512
86.12583775132539
86.13584075222566
86.14584375312594
86.15584675402621
86.16584975492648
86.17585275582674
86.18585575672701
86.19585875762729
86.20586175852756
86.21586475942783
86.2258677603281
86.23587076122837
86.24587376212864
86.25587676302891
86.26587976392918
86.27588276482945
86.28588576572972
86.29588876662999
86.30589176753027
86.31589476843054
86.3258977693308
86.33590077023106
86.34590377113133

90.407122136641
90.41712513754126
90.42712813844153
90.4371311393418
90.44713414024207
90.45713714114234
90.46714014204261
90.47714314294288
90.48714614384316
90.49714914474343
90.50715214564369
90.51715514654396
90.52715814744423
90.53716114834451
90.54716414924478
90.55716715014505
90.56717015104532
90.57717315194559
90.58717615284586
90.59717915374613
90.6071821546464
90.61718515554666
90.62718815644693
90.6371911573472
90.64719415824747
90.65719715914774
90.66720016004801
90.67720316094828
90.68720616184855
90.69720916274882
90.7072121636491
90.71721516454937
90.72721816544963
90.7372211663499
90.74722416725017
90.75722716815045
90.76723016905072
90.77723316995099
90.78723617085126
90.79723917175151
90.8072421726518
90.81724517355207
90.82724817445234
90.8372511753526
90.84725417625287
90.85725717715314
90.86726017805341
90.87726317895368
90.88726617985395
90.89726918075422
90.90727218165449
90.91727518255477
90.92727818345504
90.93728118435531
90.94728418525557
90.95728718615584
9

95.0185055516655
95.02850855256577
95.03851155346604
95.04851455436632
95.05851755526658
95.06852055616685
95.07852355706711
95.08852655796738
95.09852955886767
95.10853255976794
95.1185355606682
95.12853856156846
95.13854156246873
95.14854456336901
95.15854756426928
95.16855056516955
95.17855356606982
95.18855656697009
95.19855956787036
95.20856256877063
95.2185655696709
95.22856857057117
95.23857157147144
95.24857457237171
95.25857757327199
95.26858057417226
95.27858357507252
95.28858657597279
95.29858957687306
95.30859257777334
95.31859557867361
95.32859857957388
95.33860158047415
95.3486045813744
95.35860758227469
95.36861058317496
95.37861358407523
95.3886165849755
95.39861958587576
95.40862258677603
95.4186255876763
95.42862858857657
95.43863158947684
95.44863459037711
95.45863759127738
95.46864059217765
95.47864359307793
95.48864659397819
95.49864959487846
95.50865259577873
95.518655596679
95.52865859757928
95.53866159847955
95.54866459937982
95.55866760028009
95.56867060118034


99.64989496849054
99.65989796939083
99.6699009702911
99.67990397119135
99.68990697209162
99.69990997299189
99.70991297389217
99.71991597479244
99.72991897569271
99.73992197659298
99.74992497749324
99.75992797839352
99.76993097929379
99.77993398019406
99.78993698109433
99.7999399819946
99.80994298289487
99.81994598379514
99.8299489846954
99.83995198559568
99.84995498649594
99.85995798739621
99.8699609882965
99.87996398919677
99.88996699009704
99.89996999099729
99.90997299189756
99.91997599279784
99.92997899369811
99.93998199459838
99.94998499549865
99.95998799639892
99.96999099729919
99.97999399819946
99.98999699909973
100.0
9997 9997 9997 9997 9997


In [17]:
df_answers = pd.DataFrame()
customers = pd.Series(customers)
work_add_lat = pd.Series(work_add_lat)
work_add_lon = pd.Series(work_add_lon)
home_add_lon = pd.Series(home_add_lon)
home_add_lat = pd.Series(home_add_lat)
df_answers['customer_id'] = customers.values
df_answers['work_add_lat'] = work_add_lat.values
df_answers['work_add_lon'] = work_add_lon.values
df_answers['home_add_lat'] = home_add_lat.values
df_answers['home_add_lon'] = home_add_lon.values
df_answers.head()

,customer_id,work_add_lat,work_add_lon,home_add_lat,home_add_lon
0,00fd410f5c580c8351cafa88d82b60f3,50.809718,46.287872,50.809718,46.287872
1,0106b669d5d10148c16ea81890b9d511,57.806597,48.347270,57.806597,48.347270
2,01097e0bcfe7e25e3a1e21d53fcde472,56.114416,38.293819,56.114416,38.293819
3,0124939ab7dc640a041644cec4a97731,56.052994,38.006448,56.052994,38.006448
4,01480f0a495212838d6a963e6a3bf1c9,56.068525,37.989129,56.068525,37.989129


In [18]:
df_answers.to_csv('result_ridge.csv', index=False)

In [20]:
df_cus = final[final.customer_id == '01097e0bcfe7e25e3a1e21d53fcde472']    
print(Counter(df_cus['work_add_lat']).most_common(1)[0][0])
Counter(df_cus['work_add_lon']).most_common(1)[0][0]

56.1144161335


38.293819429708797

In [21]:
df_answers.head(10)

,customer_id,work_add_lat,work_add_lon,home_add_lat,home_add_lon
0,00fd410f5c580c8351cafa88d82b60f3,50.809718,46.287872,50.809718,46.287872
1,0106b669d5d10148c16ea81890b9d511,57.806597,48.347270,57.806597,48.347270
2,01097e0bcfe7e25e3a1e21d53fcde472,56.114416,38.293819,56.114416,38.293819
3,0124939ab7dc640a041644cec4a97731,56.052994,38.006448,56.052994,38.006448
4,01480f0a495212838d6a963e6a3bf1c9,56.068525,37.989129,56.068525,37.989129
5,0166258a6f9e7591148ad119573cd3bc,56.477934,57.534030,56.477934,57.534030
6,01903915b8823ec2122f0dc958743057,56.121155,37.965753,56.121155,37.965753
7,01cab20cbdd54e05cd41eb501dd0f6dc,56.173088,37.715271,56.173088,37.715271
8,022130a29f2165301b37dad38b9ac1c0,56.058208,38.032994,56.058208,38.032994
9,026c803388221f3d57073f1465e44f26,56.031947,38.008476,56.031947,38.008476
